In [1]:
import numpy as np
import cv2
import sklearn
import pickle

In [2]:
# face detection
face_detector_model = cv2.dnn.readNetFromCaffe('models/deploy.prototxt.txt',
                                               'models/res10_300x300_ssd_iter_140000.caffemodel')
#feature extraction
face_feature_model = cv2.dnn.readNetFromTorch('models/openface.nn4.small2.v1.t7')
#face recognition
face_recognition_model = pickle.load(open('models/machinelearning_face_person_identity2.pkl',
                                         mode='rb'))
#emotion recognition model
emotion_recognition_model = pickle.load(open('models/machinelearning_face_emotion2.pkl',
                                            mode='rb'))

In [5]:
#function to display/close images
def display(winame,image):
    cv2.namedWindow(winame)
    cv2.imshow(winame,image)
    cv2.waitKey(0) #close window when key press is detected
    cv2.destroyWindow(winame)
    cv2.waitKey(1)

In [14]:
#pipeline model test
img = cv2.imread('images_2/Youssef Al-Yakoob/IMG_2050.PNG')
image = img.copy()
h,w = img.shape[:2]
#face detection
img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop=False)
face_detector_model.setInput(img_blob)
detections = face_detector_model.forward()

if len(detections)>0:
    for i, confidence in enumerate(detections[0,0,:,2]):
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            startx,starty,endx,endy = box.astype(int)
            
            cv2.rectangle(image,(startx,starty),(endx,endy),(0,255,0))
            
            #feature extraction
            face_roi = img[starty:endy,startx:endx]
            face_blob = cv2.dnn.blobFromImage(face_roi,1/255,(96,96),(0,0,0),
                                              swapRB=True,crop=True)
            face_feature_model.setInput(face_blob)
            vectors = face_feature_model.forward()
            
            #model predictions
            #facial recognition
            face_name = face_recognition_model.predict(vectors)[0]
            face_score = face_recognition_model.predict_proba(vectors).max()
            #emotion recognition
            emotion_name = emotion_recognition_model.predict(vectors)[0]
            emotion_score = emotion_recognition_model.predict_proba(vectors).max()
            
            print(face_name)
            print(face_score)
            print(emotion_name)
            print(emotion_score)
            text_face = '{} : {:.0f}%'.format(face_name,100*face_score)
            text_emotion = '{} : {:.0f}%'.format(emotion_name,100*emotion_score)
            cv2.putText(image,text_face,(startx,starty),
                        cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
            cv2.putText(image,text_emotion,(startx,endy),
                        cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)

            

display('pipeline',image)

Youssef Al-Yakoob
0.39420213319116804
happy
0.2887989836685445


In [29]:
#pipeline function
def pipeline_model(path):
    img = cv2.imread(path)
    image = img.copy()
    h,w = img.shape[:2]
    #face detection
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop=False)
    face_detector_model.setInput(img_blob)
    detections = face_detector_model.forward()
    
    #machine learning results
    machinelearning_results = dict(face_detect_score = [],
                                   face_name = [],
                                   face_name_score = [],
                                   emotion_name = [],
                                   emotion_name_score = [],
                                   count = [])
    count = 1

    if len(detections)>0:
        for i, confidence in enumerate(detections[0,0,:,2]):
            if confidence > 0.5:
                box = detections[0,0,i,3:7]*np.array([w,h,w,h])
                startx,starty,endx,endy = box.astype(int)
            
                cv2.rectangle(image,(startx,starty),(endx,endy),(0,255,0))
            
                #feature extraction
                face_roi = img[starty:endy,startx:endx]
                face_blob = cv2.dnn.blobFromImage(face_roi,1/255,(96,96),(0,0,0),
                                                  swapRB=True,crop=True)
                face_feature_model.setInput(face_blob)
                vectors = face_feature_model.forward()
            
                #model predictions
                #facial recognition
                face_name = face_recognition_model.predict(vectors)[0]
                face_score = face_recognition_model.predict_proba(vectors).max()
                #emotion recognition
                emotion_name = emotion_recognition_model.predict(vectors)[0]
                emotion_score = emotion_recognition_model.predict_proba(vectors).max()
            

                text_face = '{} : {:.0f}%'.format(face_name,100*face_score)
                text_emotion = '{} : {:.0f}%'.format(emotion_name,100*emotion_score)
                cv2.putText(image,text_face,(startx,starty),
                            cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
                cv2.putText(image,text_emotion,(startx,endy),
                            cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
                machinelearning_results['count'].append(count)
                machinelearning_results['face_detect_score'].append(confidence)
                machinelearning_results['face_name'].append(face_name)
                machinelearning_results['face_name_score'].append(face_score)
                machinelearning_results['emotion_name'].append(emotion_name)
                machinelearning_results['emotion_name_score'].append(emotion_score)
                
                count += 1
                
    return machinelearning_results

In [30]:
results = pipeline_model('images_2/Youssef Al-Yakoob/IMG_2050.PNG')
results

{'face_detect_score': [0.9973253],
 'face_name': ['Youssef Al-Yakoob'],
 'face_name_score': [0.39420213319116804],
 'emotion_name': ['happy'],
 'emotion_name_score': [0.2887989836685445],
 'count': [1]}

In [26]:
display('pipeline',results)

In [21]:
tom =pipeline_model('images_2/Tom Curise/tom_cruise_movies.jpg')

In [22]:
display('tom',tom)